# Prediction model with Tensorflow

In [1]:
%pylab inline
import tensorflow as tf
import sys

# Include additional module
include_path = '../tensorflow_oop/'
if include_path not in sys.path:
    sys.path.append(include_path)
from tensorflow_oop.regression import *

Populating the interactive namespace from numpy and matplotlib


## Create dataset

### Load dump of dataset

In [2]:
DUMP_PATH = '../data/moscow.dump'

print('Loading dataset...')
dataset = TFDataset.load(DUMP_PATH)
print('Dataset shape: %s' % (dataset.str_shape()))

Loading dataset...
Dataset shape: 36567 : [32, 3] -> [1, 3]


### Set batch size

In [3]:
BATCH_SIZE = 32

dataset.set_batch_size(BATCH_SIZE)

### Split dataset

In [4]:
TRAIN_RATE = 0.70
VAL_RATE = 0.15
TEST_RATE = 0.15
SHUFFLE = False

print('Splitting...')
train_set, val_set, test_set = dataset.split(TRAIN_RATE, VAL_RATE, TEST_RATE, shuffle=SHUFFLE)
print('Dataset shape: %s' % dataset.str_shape())
print('Traininig  set shape: %s' % train_set.str_shape())
print('Validation set shape: %s' % val_set.str_shape())
print('Testing    set shape: %s' % test_set.str_shape())

Splitting...
Dataset shape: 36567 : [32, 3] -> [1, 3]
Traininig  set shape: 25597 : [32, 3] -> [1, 3]
Validation set shape: 5485 : [32, 3] -> [1, 3]
Testing    set shape: 5485 : [32, 3] -> [1, 3]


## Define model

In [5]:
class TFWeatherForecast(TFRegressor):
    def inference(self, inputs, **kwargs):
        # Get parameters
        input_size = self.inputs_shape[0]
        hidden_size = kwargs['hidden_size']
        rnn_count = kwargs['rnn_count']
        output_size = self.outputs_shape[-1]

        # Create RNN cells
        def rnn_cell():
            return tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
        cells = [rnn_cell() for _ in range(rnn_count)]
        multi_cell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=True)
        
        # Stack RNN layers
        batch_size = tf.shape(inputs)[0]
        init_state = multi_cell.zero_state(batch_size, dtype=tf.float32)
        rnn_outputs, last_state = tf.nn.dynamic_rnn(cell=multi_cell,
                                                    initial_state=init_state,
                                                    inputs=inputs,
                                                    swap_memory=True,
                                                    time_major=False)

        # Output layer
        with tf.variable_scope('output'):
            W = tf.Variable(tf.truncated_normal([hidden_size, output_size]),
                            name='weight')
            b = tf.Variable(tf.zeros([output_size]),
                            name='bias')
            outputs = tf.nn.xw_plus_b(last_state[-1].h, W, b)
        return tf.reshape(outputs, [-1] + self.outputs_shape)

In [6]:
LOG_DIR = '/tmp/prediction-model'
INPUTS_SHAPE = dataset.data_shape
TARGETS_SHAPE = dataset.labels_shape
OUTPUTS_SHAPE = dataset.labels_shape
HIDDEN_SIZE = 100
RNN_COUNT = 1

model = TFWeatherForecast(log_dir=LOG_DIR)
model.initialize(inputs_shape=INPUTS_SHAPE,
                 targets_shape=TARGETS_SHAPE,
                 outputs_shape=OUTPUTS_SHAPE,
                 hidden_size=HIDDEN_SIZE,
                 rnn_count=RNN_COUNT)
print(model)

Start initializing model...
Finish initializing model.
TFNeuralNetwork object:
                init: True
              loaded: False
             log_dir: /tmp/prediction-model
        inputs_shape: [32, 3]
       targets_shape: [1, 3]
       outputs_shape: [1, 3]
              inputs: Tensor("inputs:0", shape=(?, 32, 3), dtype=float32)
             targets: Tensor("targets:0", shape=(?, 1, 3), dtype=float32)
             outputs: Tensor("outputs:0", shape=(?, 1, 3), dtype=float32)
                loss: Tensor("loss:0", shape=(), dtype=float32)
         global_step: <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
                sess: <tensorflow.python.client.session.Session object at 0x7f2768442210>
      summary_writer: <tensorflow.python.summary.writer.writer.FileWriter object at 0x7f276841afd0>
    projector_config: 
              kwargs:
                   hidden_size: 100
                     rnn_count: 1
             metrics:
                   batch_train: ['loss']
   

## Train model

In [7]:
EPOCH_COUNT = 1

model.fit(train_set, epoch_count=EPOCH_COUNT, val_set=val_set)

Start training iteration...
         epoch_count: 1
          iter_count: 800
           optimizer: <class 'tensorflow.python.training.rmsprop.RMSPropOptimizer'>
       learning_rate: 0.001
  summarizing_period: 100
      logging_period: 100
   checkpoint_period: 10000
   evaluation_period: 10000
             metrics:
                   batch_train: ['all_gradients', 'loss']
              batch_validation: ['loss']
                     eval_test: []
                    eval_train: []
               eval_validation: []
                     log_train: ['loss']

Iter 100 / 800 (epoch 0 / 1):   loss = 0.017192   [1.295 sec]
Iter 200 / 800 (epoch 0 / 1):   loss = 0.008966   [1.399 sec]
Iter 300 / 800 (epoch 0 / 1):   loss = 0.005135   [1.285 sec]
Iter 400 / 800 (epoch 0 / 1):   loss = 0.006788   [1.298 sec]
Iter 500 / 800 (epoch 0 / 1):   loss = 0.005100   [1.289 sec]
Iter 600 / 800 (epoch 0 / 1):   loss = 0.001150   [1.334 sec]
Iter 700 / 800 (epoch 0 / 1):   loss = 0.000896   [1.365 sec]


## Evaluate model

In [8]:
print('Evaluating...')
if train_set is not None:
    train_eval = model.evaluate(train_set)
    print('Results on training   set: %s' % train_eval)
if val_set is not None:
    val_eval = model.evaluate(val_set)
    print('Results on validation set: %s' % val_eval)
if test_set is not None:
    test_eval = model.evaluate(test_set)
    print('Results on testing    set: %s' % test_eval)

Evaluating...
Results on training   set: {}
Results on validation set: {}
Results on testing    set: {}
